In [7]:
import nltk

In [8]:
from nltk.stem.lancaster import LancasterStemmer

In [9]:
stemmer = LancasterStemmer()

In [10]:
import numpy as np

In [11]:
import random

In [35]:
import tflearn
import tensorflow as tf

In [13]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [14]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in intent patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)  #tokenize each word
        words.extend(w) #add to words list
        documents.append((w, intent['tag'])) #add to documents
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words] #stem and lower words
words = sorted(list(set(words))) #remove duplicates
classes = sorted(list(set(classes))) #remove duplicates

In [19]:
print (len(documents), "documents", documents)
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'), (['What', 'are', 'your', 'hours', '?'], 'hours'), (['When', 'are', 'you', 'open', '?'], 'hours'), (['Which', 'mopeds', 'do', 'you', 'have', '?'], 'mopeds'), (['What', 'kinds', 'of', 'mopeds', 'are', 'there', '?'], 'mopeds'), (['What', 'do', 'you', 'rent', '?'], 'mopeds'), (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'), (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'), (['Are', 'you', 'cash', 'only', '?'], 'payments'), (['Are', 'you', 'open', 'today', '?'], 'opentoday'), (['When', 'do', 'you', 'open', 'today', '?'], 'opentoday'), (['What', 'are', 'your', 'hours',

In [31]:
#Create data
training = []
output = []

#create empty array for output
output_empty = [0] * len(classes)

#training set, bag of words for each sentence
for doc in documents: 
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
   # print(pattern_words)
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array 
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0) #(for pattern) appending 1 at the position where the word is
        
    #output is a 0 for each tag, 1 for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

#shuffle, turn into np.array
random.shuffle(training)
training = np.array(training)

# create train & test lists
train_x = list(training[: ,0]) #'bag of words' arrays, for intent pattern 
train_y = list(training[: ,1]) #'bag of words' arrays for intent class



In [38]:
# reset underlying graph data
tf.reset_default_graph()
# build neural network

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# define model & setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.00812 | time: 0.007s
| Adam | epoch: 1000 | loss: 0.00812 - acc: 1.0000 -- iter: 24/27
Training Step: 4000  | total loss: 0.00776 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.00776 - acc: 1.0000 -- iter: 27/27
--
INFO:tensorflow:/Users/aashitapatwari/Desktop/SBHacks/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [37]:
#save all data structures
import pickle
pickle.dump( {'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y }, open( "training_data", "wb"))